<a href="https://colab.research.google.com/github/brohudev/hacklytics-25/blob/main/BRCA_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [4]:
dataset = pd.read_csv('BRCA.csv')

In [5]:
list_surgerydate = []
list_lastvisit = []

list_surgerydate = dataset["Date_of_Surgery"].tolist()
for i in range(len(list_surgerydate)):
    if isinstance(list_surgerydate[i], float):
      dataset = dataset.drop(i)
    else:
      list_surgerydate[i] = datetime.strptime(str(list_surgerydate[i]), "%d-%b-%y")
list_lastvisit = dataset["Date_of_Last_Visit"].tolist()
for i in range(len(list_lastvisit)):
  if isinstance(list_lastvisit[i], float):
      dataset = dataset.drop(i)
  else:
      list_lastvisit[i] = datetime.strptime(str(list_lastvisit[i]), "%d-%b-%y")
print(list_surgerydate)
print(list_lastvisit)

[datetime.datetime(2017, 1, 15, 0, 0), datetime.datetime(2017, 4, 26, 0, 0), datetime.datetime(2017, 9, 8, 0, 0), datetime.datetime(2017, 1, 25, 0, 0), datetime.datetime(2017, 5, 6, 0, 0), datetime.datetime(2017, 9, 18, 0, 0), datetime.datetime(2017, 2, 4, 0, 0), datetime.datetime(2017, 5, 16, 0, 0), datetime.datetime(2017, 9, 28, 0, 0), datetime.datetime(2017, 2, 14, 0, 0), datetime.datetime(2017, 5, 26, 0, 0), datetime.datetime(2017, 10, 8, 0, 0), datetime.datetime(2017, 2, 24, 0, 0), datetime.datetime(2017, 6, 5, 0, 0), datetime.datetime(2017, 10, 18, 0, 0), datetime.datetime(2017, 3, 6, 0, 0), datetime.datetime(2017, 6, 15, 0, 0), datetime.datetime(2017, 10, 28, 0, 0), datetime.datetime(2017, 3, 16, 0, 0), datetime.datetime(2017, 6, 25, 0, 0), datetime.datetime(2017, 11, 7, 0, 0), datetime.datetime(2017, 3, 26, 0, 0), datetime.datetime(2017, 11, 7, 0, 0), datetime.datetime(2017, 11, 17, 0, 0), datetime.datetime(2017, 4, 5, 0, 0), datetime.datetime(2017, 11, 17, 0, 0), datetime.date

In [6]:
dataset["Date_of_Surgery"] = pd.to_datetime(dataset["Date_of_Surgery"], format="%d-%b-%y")
dataset["Date_of_Last_Visit"] = pd.to_datetime(dataset["Date_of_Last_Visit"], format="%d-%b-%y")
dataset["Days_Difference"] = (dataset["Date_of_Last_Visit"] - dataset["Date_of_Surgery"]).dt.days
dataset[["Date_of_Surgery", "Date_of_Last_Visit", "Days_Difference"]]
dataset["Patient_Status"].fillna("Dead", inplace=True)
dataset = dataset.loc[:333]


<ipython-input-6-7dfcb2347a02>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["Patient_Status"].fillna("Dead", inplace=True)


In [7]:
dataset['Tumour_Stage']=dataset['Tumour_Stage'].map({'I':1,'II':2,'III':3})
dataset['Histology']=dataset['Histology'].map({'Infiltrating Ductal Carcinoma':1,'Infiltrating Lobular Carcinoma':2,'Mucinous Carcinoma':3,})
dataset['ER status']=dataset['ER status'].map({'Positive':1})
dataset['PR status']=dataset['PR status'].map({'Positive':1})
dataset['HER2 status']=dataset['HER2 status'].map({'Positive':1,'Negative':2})
dataset['Gender']=dataset['Gender'].map({'MALE':0,'FEMALE':1})
dataset['Surgery_type']=dataset['Surgery_type'].map({'Other':1,'Modified Radical Mastectomy':2,'Lumpectomy':3,'Simple Mastectomy':4})
dataset.head(100)

,Patient_ID,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status,Days_Difference
0,TCGA-D8-A1XD,36.0,1,0.080353,0.426380,0.547150,0.273680,3,1,1,1,2,2,2017-01-15,2017-06-19,Alive,155
1,TCGA-EW-A1OX,43.0,1,-0.420320,0.578070,0.614470,-0.031505,2,3,1,1,2,3,2017-04-26,2018-11-09,Dead,562
2,TCGA-A8-A079,69.0,1,0.213980,1.311400,-0.327470,-0.234260,3,1,1,1,2,1,2017-09-08,2018-06-09,Alive,274
3,TCGA-D8-A1XR,56.0,1,0.345090,-0.211470,-0.193040,0.124270,2,1,1,1,2,2,2017-01-25,2017-07-12,Alive,168
4,TCGA-BH-A0BF,56.0,1,0.221550,1.906800,0.520450,-0.311990,2,1,1,1,2,1,2017-05-06,2019-06-27,Dead,782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,TCGA-AN-A03Y,66.0,1,-0.450770,0.423460,-0.029197,0.181920,2,2,1,1,2,2,2018-07-15,2018-07-25,Alive,10
98,TCGA-BH-A42V,41.0,1,0.100120,-0.465470,0.472370,-0.523870,1,1,1,1,1,2,2018-07-25,2019-04-23,Alive,272
100,TCGA-A2-A0CZ,46.0,1,-0.642680,0.045374,-0.204260,0.658110,2,1,1,1,1,4,2018-07-25,2018-12-11,Alive,139
101,TCGA-A2-A0D3,42.0,1,-0.175610,0.289630,1.129000,-0.382240,1,1,1,1,2,4,2018-08-04,2020-08-09,Alive,736


In [19]:
x=dataset[["Age","Gender","Protein1","Protein2","Protein3","Protein4","Tumour_Stage","Histology","ER status","PR status","HER2 status","Surgery_type","Days_Difference"]]
y=dataset["Patient_Status"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [20]:
model = SVC()
model.fit(x_train,y_train)
y_predicted=model.predict(x_test)
scores=cross_val_score(model, x, y, cv=5)
accuracy_score(y_test,y_predicted)

0.9

In [21]:
import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model saved successfully as 'model.pkl'")

Model saved successfully as 'model.pkl'
